### **Chapter 3: Neural Contextual Bandits: Generalization Through Shared Representation**

#### **3.1 The Limits of Disjoint Models and the Need for Generalization**

In our exploration thus far, we have journeyed through two fundamental paradigms of machine learning for recommendation. In Chapter 1, we constructed a `MLPRecommender`, a model endowed with the expressive power of deep learning. It learned rich, non-linear relationships from a large, static dataset, creating effective, generalizable embeddings for users and products. Its crucial flaw, however, was its static nature; it was a creature of the past, unable to adapt to new information without complete retraining.

In Chapter 2, we addressed this inertia by introducing the `LinUCBAgent`, an online learning system grounded in the contextual bandit framework. This agent, learning at every interaction, demonstrated a remarkable ability to adapt and optimize its strategy over time, ultimately outperforming its static counterpart. Yet, it too possessed a critical architectural weakness. Our `LinUCBAgent` was a *disjoint model*. It maintained an independent set of parameters—a matrix $A_a$ and vector $b_a$—for each of the 50 products in our catalog.

Consider the implications of this design. When the agent recommends "Premium Dog Toy A" and observes a click, it updates only the parameters for that specific toy. If it is subsequently asked to evaluate "Deluxe Dog Toy B," it approaches the problem with no transference of knowledge. The insight that this user has an affinity for dog toys remains siloed within the parameters of "Toy A." The model cannot **generalize**.

This is not merely inefficient; it is fundamentally unscalable. In a real-world e-commerce setting with millions of products, maintaining a separate model for each item is computationally infeasible and statistically disastrous. The vast majority of items would be recommended so infrequently that their parameters would never be reliably estimated—a severe form of the cold-start problem.

The path forward must therefore be a synthesis. We need a model that combines the adaptive, principled exploration of a bandit algorithm with the powerful, generalizable representation learning of a neural network. We seek a single, unified model that learns from every interaction to refine a shared understanding of the world, where learning about one dog toy informs its beliefs about *all* dog toys. This chapter is dedicated to the theory and implementation of such a model: the **Neural Bandit**.

#### **3.2 Generalizing the UCB Principle Beyond Linearity**

To build our new agent, we must first return to the theoretical foundation of the UCB algorithm and abstract it away from the restrictive assumption of linearity.

**Recap: The LinUCB Assumption**

Recall from Chapter 2 that the LinUCB algorithm operates on a crucial assumption: the expected reward $r_{t,a}$ for a given arm $a$ at time $t$ is a linear function of its context feature vector $x_{t,a}$.

**Definition 3.1: Linear Reward Model**
Let $x_{t,a} \in \mathbb{R}^d$ be the feature vector for arm $a$ at time $t$. The LinUCB model assumes there exists an unknown but fixed parameter vector $\theta_a^* \in \mathbb{R}^d$ such that the expected reward is:
$$
\mathbb{E}[r_{t,a} | x_{t,a}] = (x_{t,a})^T \theta_a^*
$$

From this assumption, we derived the UCB score, which was the sum of two terms: the *estimated reward* based on our learned $\hat{\theta}_a$, and an *exploration bonus* proportional to the uncertainty in that estimate.

$$
\text{score}(a) = \underbrace{(x_{t,a})^T \hat{\theta}_{t-1,a}}_{\text{Exploitation}} + \underbrace{\alpha \sqrt{(x_{t,a})^T A_{t-1,a}^{-1} x_{t,a}}}_{\text{Exploration}}
$$

**Definition 3.2: LinUCB Payoff Score (Recap)**
At the beginning of time step $t$, using parameters $(A_{t-1,a}, b_{t-1,a})$ estimated from all data up to step $t-1$, the payoff for arm $a$ is:
$$
p_{t,a} = \underbrace{ (x_{t,a})^T \hat{\theta}_{t-1,a} }_{\text{Exploitation}} + \underbrace{ \alpha \sqrt{ (x_{t,a})^T A_{t-1,a}^{-1} x_{t,a} } }_{\text{Exploration}}
$$
where $\hat{\theta}_{t-1,a} = A_{t-1,a}^{-1} b_{t-1,a}$.

The brilliance of this formulation lies in its elegant, closed-form expression for uncertainty, derived from the properties of ridge regression.

**The Generalization Step**

Our goal is to replace the linear model $(x_{t,a})^T \theta_a$ with a more powerful, non-linear function approximator, which we will represent as $f(x_{t,a})$. Let us imagine, for now, that this $f$ is an arbitrary function—in our case, it will be a neural network.

The core UCB principle is not intrinsically tied to linearity. It is a general strategy for decision-making under uncertainty. We can state it more broadly:

**Principle: The Generalized UCB**
At each time step $t$, for each arm $a$, select the arm that maximizes the following payoff:
$$
\text{select } a_t = \arg\max_{a \in \mathcal{A}} \left( \hat{\mu}_{t-1}(x_{t,a}) + \kappa_{t-1}(x_{t,a}) \right)
$$
where:
*   $\hat{\mu}_{t-1}(x_{t,a})$ is the model's estimate of the mean reward for arm $a$ given features $x_{t,a}$, based on information up to step $t-1$.
*   $\kappa_{t-1}(x_{t,a})$ is the upper confidence bound, or "exploration bonus," which quantifies the uncertainty of the estimate $\hat{\mu}$. It should be large when the model is uncertain and small when it is confident.

This generalization presents us with a formidable new challenge. For linear models, the confidence bound $\kappa$ can be derived analytically. But how do we compute a meaningful confidence bound for the output of a complex, non-linear model like a deep neural network? This is the central question that the NeuralUCB algorithm aims to answer.

#### **3.3 NeuralUCB: Confidence Bounds via Gradient-based Approximation**

The NeuralUCB algorithm provides an elegant and effective method for applying the UCB principle to neural network models. It acknowledges that while we cannot find an exact, closed-form confidence bound for the network's output, we can construct a powerful approximation by considering the network's behavior in the vicinity of its current parameterization.

The core idea is to use a form of online ridge regression, not on the input features $x$ directly, but on the *gradient of the network's output with respect to its parameters*. This gradient acts as a high-dimensional, learned feature representation that captures the sensitivity of the model's prediction to changes in its weights.

Let us formalize this.

**Definition 3.2: The NeuralUCB Model**
The NeuralUCB agent is defined by:
1.  A neural network $g(x; \theta)$ with parameters $\theta \in \mathbb{R}^p$, where $p$ is the total number of trainable parameters in the network. This network takes a context feature vector $x$ as input and outputs a scalar prediction of the reward.
2.  A shared $p \times p$ matrix $A$ and a shared $p \times 1$ vector $b$, which are analogous to the parameters in LinUCB but are now shared across all arms.

Initially, at time $t=0$, we have:
$$
A_0 = \lambda I_p \quad \text{(where } \lambda > 0 \text{ is a regularization parameter)}
$$
$$
b_0 = \mathbf{0}_{p \times 1} \quad \text{(a zero vector of size p)}
$$

**The Prediction and Update Mechanism**

At each time step $t$, the agent performs the following steps:

1.  **Observe Context:** For the current user, the agent has access to a set of feature vectors $\{x_{t,a}\}_{a \in \mathcal{A}}$, one for each potential arm (product).
2.  **Estimate Payoff:** For each arm $a$, the agent calculates a payoff $p_{t,a}$. This requires two components:
    *   **a. Network Prediction (Exploitation):** First, it computes the network's current estimate of the reward, $g(x_{t,a}; \hat{\theta}_{t-1})$, where $\hat{\theta}_{t-1}$ are the network parameters estimated using data up to the previous step.
    *   **b. Confidence Bound (Exploration):** It then computes the gradient of the network's output with respect to the parameters, evaluated at the current parameter estimate:
        $$
        \nabla g_{t,a} = \nabla_{\theta} g(x_{t,a}; \hat{\theta}_{t-1})
        $$
        This gradient vector $\nabla g_{t,a} \in \mathbb{R}^p$ is treated as the effective feature vector for the confidence bound calculation. The full payoff is:
        $$
        p_{t,a} = \underbrace{g(x_{t,a}; \hat{\theta}_{t-1})}_{\text{Exploitation}} + \underbrace{\alpha \sqrt{ (\nabla g_{t,a})^T A_{t-1}^{-1} (\nabla g_{t,a}) }}_{\text{Exploration}}
        $$
        where $\alpha \ge 0$ is the exploration hyperparameter.

3.  **Select Arm:** The agent chooses the arm with the highest payoff:
    $$
    a_t = \arg\max_{a \in \mathcal{A}} p_{t,a}
    $$

4.  **Observe Reward & Update:** The agent plays arm $a_t$, observes the real-world reward $r_t$, and then updates its shared parameters:
    *   Update the evidence matrix $A$ and reward vector $b$:
        $$
        A_t = A_{t-1} + (\nabla g_{t,a_t}) (\nabla g_{t,a_t})^T
        $$
        $$
        b_t = b_{t-1} + r_t (\nabla g_{t,a_t})
        $$
    *   Update the network's weights $\theta$. This is typically done by performing one or more steps of gradient descent on a loss function. A common choice is the squared error between the prediction and the observed reward, potentially combined with a ridge penalty:
        $$
        \mathcal{L}(\theta) = (g(x_{t,a_t}; \theta) - r_t)^2 + \lambda ||\theta||_2^2
        $$
        The parameters $\hat{\theta}_{t-1}$ are updated to $\hat{\theta}_t$ by optimizing this loss on the newly acquired data point $(x_{t,a_t}, r_t)$.

**Remark 3.1: The Power of Shared Parameters**
The significance of this formulation cannot be overstated. We now have a **single, shared** matrix $A$ and vector $b$ for the entire system. When the agent recommends product $a_t$ and receives reward $r_t$, the subsequent update to $A_t$, $b_t$, and $\hat{\theta}_t$ refines a global model of the world. This new knowledge is immediately available for evaluating *all other products*, even those in completely different categories. The model learns to generalize across the entire item space, elegantly solving the primary weakness of the disjoint LinUCB model. This architecture is both statistically efficient and computationally scalable.

Now, let us proceed to implement this more sophisticated agent. We will begin by defining the neural network architecture that will serve as the core of our `NeuralUCBAgent`.

### **3.4 Implementing the Neural Network Approximator**

The text concludes by setting up our next task: implementing the agent. Before we can build the full `NeuralUCBAgent`, we must first construct its heart: the neural network $g(x; \theta)$ that will act as our non-linear function approximator. Let's call this the `NeuralBanditNetwork`.

#### **3.4.1 Architectural Considerations**

**Intuition First:** Our goal for this network is to be a general-purpose learner. It must be powerful enough to capture complex, non-linear relationships between user and item features, yet simple enough for efficient online training where updates happen one data point at a time. We will employ a standard Multi-Layer Perceptron (MLP) architecture, similar in spirit to the one we built in Chapter 1, but with a crucial difference.

Unlike our previous `MLPRecommender` which took discrete user and item *IDs* as input and learned embeddings from scratch, this new network will operate on rich *feature vectors*. This is the key design choice that allows it to generalize. By learning a function on the feature space itself, the model can make predictions for any user-item pair, as long as we can describe them with features. Learning that a user who likes "Dog Toy" features also responds well to a specific new product with "Dog Toy" features is now not only possible, but is the central mechanism of the model.

We will design a simple MLP with two hidden layers and ReLU activation functions. The final output will be a single scalar, representing the network's estimate of the reward.

**Formalizing the Implementation:** Let us now define the network in PyTorch. We will construct a class `NeuralBanditNetwork` that inherits from `torch.nn.Module`.

**Code as a Didactic Tool: Dissection**

Here is the structure of our network class. We will examine it piece by piece.

```python
# Introduce the core components first
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List

class NeuralBanditNetwork(nn.Module):
    def __init__(self, feature_dim: int, hidden_dims: List[int] = [128, 64]):
        super(NeuralBanditNetwork, self).__init__()
        # ... layer definitions ...

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # ... forward pass logic ...
```

Now, let us dissect the two primary methods, `__init__` and `forward`.

**The `__init__` method:** This is the constructor for our module, where we define the layers of our network.

*   `super(NeuralBanditNetwork, self).__init__()`: This is standard boilerplate to call the constructor of the parent class, `nn.Module`.
*   `self.layers = nn.ModuleList()`: We use a `ModuleList` to hold our sequence of layers. This is a convenient way to manage a variable number of layers.
*   The `for` loop constructs the network dynamically based on the `hidden_dims` list.
    *   `self.layers.append(nn.Linear(input_dim, output_dim))`: For each step, it creates a `nn.Linear` layer. The first layer maps the `feature_dim` to the size of the first hidden layer. Subsequent layers map the previous hidden size to the next one.
*   `self.output_layer = nn.Linear(hidden_dims[-1], 1)`: The final layer maps the last hidden dimension to a single scalar output.

**Remark 3.2: Absence of a Final Sigmoid.**
You will note that unlike in Chapter 1, we do **not** apply a sigmoid activation function to the final output. This is a deliberate and important choice. The UCB calculation relies on the properties of a linear model approximation. The raw output of the linear layer (a logit) is better suited for this than the bounded, non-linear output of a sigmoid. Forcing the output between 0 and 1 can squash the gradients and interfere with the delicate confidence bound estimation. We are predicting a raw score, not a probability.

**The `forward` method:** This method defines the computational graph—how data flows through the network.

*   `for layer in self.layers:`: It iterates through the hidden layers defined in our `ModuleList`.
*   `x = F.relu(layer(x))`: For each layer, it performs the linear transformation and then applies the Rectified Linear Unit (ReLU) activation function. This introduces the crucial non-linearity that allows the network to learn complex patterns.
*   `output = self.output_layer(x)`: Finally, it passes the result from the last hidden layer through the output layer to produce the single scalar prediction.

**Assembling the Full Module**

Putting this all together, the complete, self-contained, and runnable code for our network module is as follows.

```python
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List

class NeuralBanditNetwork(nn.Module):
    """
    A simple Multi-Layer Perceptron (MLP) to act as the function approximator
    for the NeuralUCB agent.

    The network takes a feature vector of a given dimension and outputs a single
    scalar value representing the predicted reward.
    """
    def __init__(self, feature_dim: int, hidden_dims: List[int] = [128, 64]):
        """
        Initializes the network layers.

        Args:
            feature_dim (int): The dimensionality of the input feature vector.
            hidden_dims (List[int]): A list of integers, where each integer is the
                                     size of a hidden layer.
        """
        super(NeuralBanditNetwork, self).__init__()
        
        # We use a ModuleList to hold our sequence of layers
        self.layers = nn.ModuleList()
        
        # Define the input layer
        input_dim = feature_dim
        
        # Define the hidden layers dynamically
        for h_dim in hidden_dims:
            self.layers.append(nn.Linear(input_dim, h_dim))
            input_dim = h_dim # The next layer's input is the current layer's output
            
        # Define the output layer
        self.output_layer = nn.Linear(hidden_dims[-1], 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Defines the forward pass of the network.

        Args:
            x (torch.Tensor): The input tensor of shape (batch_size, feature_dim).

        Returns:
            torch.Tensor: The output tensor of shape (batch_size, 1).
        """
        # Pass through the hidden layers with ReLU activation
        for layer in self.layers:
            x = F.relu(layer(x))
            
        # Pass through the output layer (no activation)
        output = self.output_layer(x)
        return output

```

With this `NeuralBanditNetwork` defined, we now have the core component $g(x; \theta)$. Our next step is to build the agent class, `NeuralUCBAgent`, which will instantiate this network and implement the full predict-and-update cycle described in Section 3.3.

### **3.5 Implementing the NeuralUCBAgent**

With our `NeuralBanditNetwork` defined, we can now assemble the complete agent. This class, `NeuralUCBAgent`, will be the orchestrator. It will hold an instance of the network, manage the UCB-specific parameters ($A$ and $b$), and implement the core `predict` and `update` logic that defines the NeuralUCB algorithm.

#### **3.5.1 The Agent's Architecture**

**Intuition First:** The `NeuralUCBAgent` class is the brain of our operation. It must contain all the necessary components to execute the algorithm described in Section 3.3. Let's think about what it needs to store and what it needs to do.

1.  **The Function Approximator:** It needs an instance of our `NeuralBanditNetwork`, which we will call `self.model`. This is the $g(x; \theta)$ from our theory.
2.  **The UCB Machinery:** It needs to store the shared matrix $A$ and vector $b$. These will be PyTorch tensors to integrate seamlessly with the rest of our PyTorch-based workflow. $A$ will be a $p \times p$ matrix and $b$ a $p \times 1$ vector, where $p$ is the total number of parameters in `self.model`.
3.  **The Training Mechanism:** Since we update the network's parameters $\theta$ at each step, the agent needs its own optimizer (we will use `Adam`) and a loss function (we will use Mean Squared Error, or `MSELoss`).
4.  **Hyperparameters:** The agent must store the key hyperparameters that control its behavior: the exploration factor $\alpha$ and the regularization parameter $\lambda$.

The agent's life cycle will be a loop:
*   At prediction time, it will use the network for the exploitation term and the UCB machinery ($A$ and the gradients) for the exploration term to select an arm.
*   At update time, it will use the observed reward to update both the UCB machinery *and* the network's parameters via one step of gradient descent.

**Formalizing the Implementation: The `__init__` Method**

Let us begin by constructing the agent's `__init__` method. This is where we will initialize all the components listed above.

**Code as a Didactic Tool: Dissecting the Constructor**

```python
# Introduce the constructor first
import torch
import torch.optim as optim
from typing import List

class NeuralUCBAgent:
    def __init__(self,
                 feature_dim: int,
                 n_arms: int,
                 hidden_dims: List[int] = [128, 64],
                 lambda_: float = 1.0,
                 alpha: float = 1.0,
                 lr: float = 0.01):
        # ... initialization logic ...
```

Now, let's dissect this constructor line by line.

*   `model = NeuralBanditNetwork(feature_dim, hidden_dims)`: We instantiate our neural network.
*   `self.p = sum(p.numel() for p in self.model.parameters() if p.requires_grad)`: This is a crucial step. We calculate $p$, the total number of trainable parameters in our network. We iterate through all parameters in `self.model.parameters()` and sum up their number of elements (`p.numel()`). This gives us the dimensionality for our UCB machinery.
*   `self.A = torch.eye(self.p) * lambda_`: We initialize the matrix $A$. As per the algorithm definition, it starts as a $p \times p$ identity matrix, scaled by the regularization hyperparameter $\lambda$. `torch.eye(self.p)` creates the identity matrix.
*   `self.b = torch.zeros((self.p, 1))`: We initialize the vector $b$ as a zero vector of size $p \times 1$.
*   `self.optimizer = optim.Adam(self.model.parameters(), lr=lr)`: We set up the Adam optimizer, telling it which parameters it is responsible for optimizing (`self.model.parameters()`) and what learning rate (`lr`) to use.
*   `self.loss_fn = nn.MSELoss()`: We define our loss function. The Mean Squared Error, $(y_{pred} - y_{true})^2$, is a natural choice for regressing onto a continuous reward signal.

**Assembling the `__init__` Method**

Here is the complete constructor with explanations integrated as comments.

```python
class NeuralUCBAgent:
    """
    Implements the NeuralUCB algorithm.

    This agent uses a neural network to approximate the reward function and
    calculates an upper confidence bound based on the gradient of the network's
    output with respect to its parameters.
    """
    def __init__(self,
                 feature_dim: int,
                 n_arms: int,
                 hidden_dims: List[int] = [128, 64],
                 lambda_: float = 1.0,
                 alpha: float = 1.0,
                 lr: float = 0.01):
        """
        Initializes the agent's components.

        Args:
            feature_dim (int): Dimensionality of the input context features.
            n_arms (int): The number of arms (actions) available.
            hidden_dims (List[int]): Sizes of the hidden layers for the network.
            lambda_ (float): Regularization parameter for the A matrix.
            alpha (float): Exploration-exploitation trade-off parameter.
            lr (float): Learning rate for the network's optimizer.
        """
        self.feature_dim = feature_dim
        self.n_arms = n_arms
        self.alpha = alpha
        self.lambda_ = lambda_ # Store for potential future use

        # 1. The Neural Network Function Approximator
        self.model = NeuralBanditNetwork(feature_dim, hidden_dims)

        # 2. The UCB Machinery
        # Get the total number of trainable parameters in the model
        self.p = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        
        # A is a (p x p) matrix, initialized to lambda * Identity
        self.A = torch.eye(self.p) * self.lambda_
        # b is a (p x 1) vector, initialized to zeros
        self.b = torch.zeros((self.p, 1))

        # 3. The Training Mechanism
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.loss_fn = nn.MSELoss()
```

#### **3.5.2 The Prediction Mechanism**

Now we arrive at the core of the algorithm: the `predict` method. This method must calculate the UCB score for every arm and select the best one.

**Intuition First:** For each arm $a$, we need to compute $p_{t,a} = g(x_{t,a}) + \alpha \sqrt{ (\nabla g_{t,a})^T A^{-1} (\nabla g_{t,a}) }$. This involves two distinct steps for each arm: a forward pass through the network to get the exploitation term $g(x_{t,a})$, and a more complex calculation involving gradients and matrix inversion for the exploration term.

**Formalizing the Implementation: The `predict` Method**

Let's break down the required steps within the `predict` method.

1.  **Prepare for Batch Prediction:** We will receive a list or tensor of feature vectors, one for each arm. Let's call this `feature_vectors`.
2.  **Calculate Exploitation Term:** We can compute the predicted rewards for all arms in a single batch by passing `feature_vectors` through `self.model`. This is efficient.
3.  **Calculate Exploration Term (Per Arm):** This is the most intricate part. For each arm $a$:
    a.  We need the gradient $\nabla_{\theta} g(x_{t,a}; \hat{\theta})$. In PyTorch, this is not part of a standard forward pass. We must compute it explicitly. The function `torch.autograd.grad` is designed for this. It computes the gradient of a scalar output (our network's prediction for one arm) with respect to the model's parameters.
    b.  We must flatten this gradient into a single $p \times 1$ vector to match the dimensions of $A$ and $b$.
    c.  We need to compute $A^{-1}$. We will use `torch.inverse(self.A)`.
    d.  We perform the quadratic form multiplication: $(\nabla g)^T A^{-1} (\nabla g)$.
    e.  We take the square root and multiply by $\alpha$.
4.  **Combine and Select:** We add the exploitation and exploration terms for each arm to get the final UCB scores, and then select the arm with the highest score using `torch.argmax`.

**Code as a Didactic Tool: Dissecting the `predict` Method**

```python
def predict(self, feature_vectors: torch.Tensor) -> int:
    """
    Selects an arm based on the NeuralUCB scoring rule.

    Args:
        feature_vectors (torch.Tensor): A tensor of shape (n_arms, feature_dim)
                                        containing the context for each arm.

    Returns:
        int: The index of the chosen arm.
    """
    # Ensure model is in evaluation mode for prediction
    self.model.eval()

    # We will store gradients for the update step to avoid re-computation
    self.arm_gradients = []
    ucb_scores = torch.zeros(self.n_arms)

    # Pre-calculate the inverse of A for efficiency in the loop
    A_inv = torch.inverse(self.A)

    # --- Step 1: Calculate Exploitation Term for all arms in a batch ---
    # The model's raw prediction is the exploitation score
    with torch.no_grad(): # No need to track gradients for this part
        exploit_scores = self.model(feature_vectors).squeeze()

    # --- Step 2 & 3: Calculate Exploration Term for each arm ---
    for i in range(self.n_arms):
        x_i = feature_vectors[i].unsqueeze(0) # Shape: (1, feature_dim)
        
        # We need to compute gradients, so we clear any old ones
        self.model.zero_grad()
        
        # Forward pass for a single arm to get a scalar output
        pred_i = self.model(x_i)
        
        # Calculate the gradient of the output w.r.t. model parameters
        # This is the \nabla g_{t,a} term
        grads = torch.autograd.grad(pred_i, self.model.parameters())
        
        # Flatten the gradients into a single vector of size (p, 1)
        g_i = torch.cat([g.view(-1) for g in grads]).view(-1, 1)
        self.arm_gradients.append(g_i) # Save for the update step

        # Calculate the exploration bonus: alpha * sqrt(g_i^T * A_inv * g_i)
        explore_bonus = self.alpha * torch.sqrt(g_i.T @ A_inv @ g_i)
        
        # --- Step 4: Combine and store score ---
        ucb_scores[i] = exploit_scores[i] + explore_bonus
        
    # Select the arm with the highest UCB score
    chosen_arm = torch.argmax(ucb_scores).item()
    return chosen_arm
```

**Remark 3.3: A Note on Efficiency.**
In the code above, we loop through each arm to calculate its gradient. While correct, this can be computationally intensive if the number of arms is very large. Advanced implementations might use techniques like Jacobian-vector products to speed this up, but for initial try and test solution, this explicit loop is superior as it perfectly mirrors the per-arm formula from the theory. We also save the computed gradients in `self.arm_gradients` to reuse in the `update` step, avoiding redundant computation.

### **3.6 A Pedagogical Deep Dive: Scaling Exploration to Millions of Arms**

The `predict` method we developed in Section 3.5.2 is correct, but it contains a hidden computational bottleneck that renders it impractical for real-world systems with large action spaces. This section is dedicated to dissecting this bottleneck, understanding its mathematical structure, and implementing a vastly more efficient, vectorized solution.

#### **3.6.1 The "K-Pass Problem": Analyzing the Naive Loop**

**Intuition First:** Let us reconsider the loop at the heart of our `predict` method:

```python
# The naive loop from our previous implementation
for i in range(self.n_arms):
    # ...
    grads = torch.autograd.grad(pred_i, self.model.parameters())
    # ...
```

The call to `torch.autograd.grad` is the workhorse of automatic differentiation in PyTorch. To compute the gradient of a scalar output with respect to all model parameters, it essentially performs a **backward pass** through the computation graph, propagating gradients from the output back to the parameters using the chain rule.

Our loop iterates `n_arms` times. Let us denote the number of arms by $K$. This means that to select a single arm, our agent performs **one forward pass** (to get all exploitation scores) and **$K$ separate backward passes** (one for each arm's gradient). This is what we shall call the **K-Pass Problem**. If our e-commerce site has $K=50,000$ products, this approach would require 50,000 backward passes just to make one recommendation. This is not merely inefficient; it is computationally non-viable.

Our goal is to reformulate the exploration bonus calculation to eliminate this loop, leveraging the power of batched matrix operations.

#### **3.6.2 The Vectorized Solution via the Jacobian Matrix**

**Formalizing with Unyielding Rigor:** The key to a batched computation lies in understanding the mathematical object that our loop was constructing piece by piece.

**Definition 3.3: The Output-Parameter Jacobian**
Let our neural network be the function $g(x; \theta)$, where $x$ is an input feature vector and $\theta \in \mathbb{R}^p$ is the vector of all $p$ model parameters. When we present a batch of $K$ feature vectors, $\{x_1, x_2, \dots, x_K\}$, the network produces a vector of $K$ scalar reward predictions, $\hat{\mathbf{r}} \in \mathbb{R}^K$, where $\hat{r}_i = g(x_i; \theta)$.

The **Jacobian** of the network's output vector $\hat{\mathbf{r}}$ with respect to the parameter vector $\theta$ is a $K \times p$ matrix, denoted by $J$, where each entry $J_{ij}$ is the partial derivative of the $i$-th output with respect to the $j$-th parameter:
$$
J_{ij} = \frac{\partial \hat{r}_i}{\partial \theta_j}
$$
The $i$-th row of this Jacobian, $J_{i,:}$, is therefore the transpose of the gradient vector $\nabla_{\theta} g(x_i; \theta)$ that we were calculating inside our loop.
$$
J =
\begin{pmatrix}
\frac{\partial \hat{r}_1}{\partial \theta_1} & \frac{\partial \hat{r}_1}{\partial \theta_2} & \cdots & \frac{\partial \hat{r}_1}{\partial \theta_p} \\
\frac{\partial \hat{r}_2}{\partial \theta_1} & \frac{\partial \hat{r}_2}{\partial \theta_2} & \cdots & \frac{\partial \hat{r}_2}{\partial \theta_p} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial \hat{r}_K}{\partial \theta_1} & \frac{\partial \hat{r}_K}{\partial \theta_2} & \cdots & \frac{\partial \hat{r}_K}{\partial \theta_p}
\end{pmatrix}
=
\begin{pmatrix}
— (\nabla_{\theta} g(x_1; \theta))^T — \\
— (\nabla_{\theta} g(x_2; \theta))^T — \\
\vdots \\
— (\nabla_{\theta} g(x_K; \theta))^T —
\end{pmatrix}
$$

Now, let us rewrite the exploration bonus for a single arm $i$ using this notation. Let $g_i = \nabla_{\theta} g(x_i; \theta)$ be the gradient (a column vector of size $p \times 1$). The squared exploration bonus is $(g_i)^T A^{-1} g_i$. This is exactly the term $(J_{i,:}) A^{-1} (J_{i,:})^T$.

How can we compute this for all $K$ arms at once? Consider the matrix product $M = J A^{-1} J^T$.

*   $J$ has dimensions $(K \times p)$.
*   $A^{-1}$ has dimensions $(p \times p)$.
*   $J^T$ has dimensions $(p \times K)$.
*   The resulting matrix $M$ has dimensions $(K \times K)$.

Let's examine the element $M_{ii}$ on the diagonal of this matrix.
$$
M_{ii} = (\text{i-th row of } J) \cdot (\text{i-th column of } A^{-1} J^T)
$$
By the rules of matrix multiplication, the $i$-th column of $A^{-1} J^T$ is $A^{-1}$ multiplied by the $i$-th column of $J^T$. The $i$-th column of $J^T$ is simply the vector $g_i$.
$$
M_{ii} = (J_{i,:}) \cdot (A^{-1} g_i) = (g_i)^T A^{-1} g_i
$$

**Theorem 3.1: Batched Exploration Bonus Calculation**
The squared exploration bonuses for all $K$ arms are the diagonal elements of the matrix product $J A^{-1} J^T$, where $J$ is the Jacobian of the network outputs with respect to the parameters.
$$
\left[ (\text{bonus}_1)^2, \dots, (\text{bonus}_K)^2 \right] = \text{diag} \left( J A^{-1} J^T \right)
$$

This is our breakthrough. If we can compute the entire Jacobian $J$ efficiently, we can replace the $K$-Pass loop with a series of highly optimized matrix multiplications.

#### **3.6.3 Implementation "From Scratch" with Modern PyTorch**

**Code as a Didactic Tool:** How do we compute the full Jacobian $J$ in PyTorch without a loop? The standard `torch.autograd.grad` is not designed for this; it computes Vector-Jacobian products. However, the modern functional API, `torch.func`, provides exactly the tool we need: `vmap` (vectorized map).

`vmap` is a function transformer. It takes a function that operates on a single sample and returns a new function that operates on a batch of samples, handling the batching dimension automatically. We can write a simple function to get the gradient for one arm, and `vmap` will "lift" it into a fully vectorized version that computes the Jacobian for all arms at once.

Let's build a new, efficient `predict_vectorized` method.

**Dissection of the Vectorized `predict` Method**

1.  **Define a per-sample gradient function:** We'll define a small helper function `get_grad` that takes a single feature vector and computes the flattened gradient vector, just as we did inside the loop before.
2.  **Vectorize it with `vmap`:** We will use `torch.func.vmap(get_grad)` to create a new function, `batched_get_grad`, which will take the entire batch of feature vectors and return the full Jacobian matrix $J$.
3.  **Perform matrix operations:** We will implement the formula from Theorem 3.1. A particularly efficient way to compute `diag(J @ A_inv @ J.T)` is with `torch.einsum`, which allows for expressive and optimized tensor contractions. The expression `torch.einsum('ij,ji->i', J @ A_inv, J.T)` computes the matrix product and extracts the diagonal in a single, highly optimized step.

**Assembling the New Method**

First, ensure you have the necessary functional tools. In recent PyTorch versions, this is `torch.func`.

```python
# We need vmap and functional_call from torch.func
from torch.func import vmap, functional_call

# This new method will replace our old 'predict' method in the NeuralUCBAgent class.
def predict_vectorized(self, feature_vectors: torch.Tensor) -> int:
    """
    Selects an arm using a vectorized calculation of the NeuralUCB score.
    This method avoids explicit loops over arms for gradient calculation.

    Args:
        feature_vectors (torch.Tensor): Shape (n_arms, feature_dim).

    Returns:
        int: The index of the chosen arm.
    """
    self.model.eval()
    n_arms = feature_vectors.shape[0]

    # --- Step 1: Exploitation Term (same as before) ---
    exploit_scores = self.model(feature_vectors).squeeze()

    # --- Step 2: Vectorized Exploration Term ---
    
    # Pre-calculate the inverse of A
    A_inv = torch.inverse(self.A)

    # We need to define a function that computes the gradient for a *single* sample.
    # It must be self-contained, so we pass model params and buffers to it.
    params = {k: v.detach() for k, v in self.model.named_parameters()}
    buffers = {k: v.detach() for k, v in self.model.named_buffers()}

    def compute_grad(x_single):
        # Use functional_call to run the model with specific params/buffers
        # on a single input x_single, which has shape (feature_dim,)
        pred = functional_call(self.model, (params, buffers), args=(x_single.unsqueeze(0),))
        
        # Compute gradient of this single prediction w.r.t. parameters
        grads = torch.autograd.grad(pred, params.values())
        
        # Flatten and concatenate into a single vector
        return torch.cat([g.view(-1) for g in grads])

    # Use vmap to vectorize our compute_grad function.
    # It will now accept a batch of feature vectors and return a batch of gradients (the Jacobian).
    # The result, J, will have shape (n_arms, p).
    J = vmap(compute_grad)(feature_vectors)
    
    # Save the Jacobian for the update step. Note its shape.
    self.last_jacobian = J

    # --- Step 3: Batched bonus calculation using einsum ---
    # This is the efficient implementation of diag(J @ A_inv @ J.T)
    # 'ij,ji->i' means: multiply matrix J@A_inv (ij) with J.T (ji)
    # and sum over the common dimension j, keeping only the diagonal elements (i).
    bonus_squared = torch.einsum('ij,ji->i', J @ A_inv, J.T)
    
    # Clamp to avoid numerical issues with negative values from floating point errors
    explore_bonuses = self.alpha * torch.sqrt(torch.clamp(bonus_squared, min=0))

    # --- Step 4: Combine and Select ---
    ucb_scores = exploit_scores + explore_bonuses
    chosen_arm = torch.argmax(ucb_scores).item()
    
    return chosen_arm
```
**Remark 3.4: A Note on the Update Step.**
Our `update` method will also need to be adjusted. It previously relied on a single gradient vector `g_i`. With this new `predict_vectorized` method, we have the full Jacobian `J` available. The update will now use the specific row of the Jacobian corresponding to the arm that was chosen, e.g., `g_t = self.last_jacobian[chosen_arm].view(-1, 1)`. This avoids re-computing any gradients.

#### **3.6.4 Analysis: The Scalability Trade-off**

We have replaced a method that was slow in computation time with one that is fast. However, we must always analyze the resources we are using, particularly memory.

**Naive Loop (K-Pass) Method**
*   **Pros:**
    *   **Low Memory:** Requires storing only one gradient vector ($p \times 1$) at a time. Memory usage is independent of the number of arms $K$.
    *   **Simple to Implement:** The logic directly mirrors the mathematical formula for a single arm.
*   **Cons:**
    *   **Computationally Slow:** Time complexity is roughly $O(K \cdot C_{backward})$, where $C_{backward}$ is the cost of a single backward pass. This scales linearly and poorly with the number of arms.

**Vectorized Jacobian Method**
*   **Pros:**
    *   **Computationally Fast:** Leverages highly optimized BLAS/cuBLAS libraries for matrix operations. The time complexity is dominated by the Jacobian computation and matrix multiplies, but the constant factors are much smaller, leading to massive speedups in practice.
*   **Cons:**
    *   **High Memory:** The primary drawback. It requires instantiating the full Jacobian matrix $J$ in memory. The size of this matrix is $K \times p$. For a system with $K=1,000,000$ arms and a network with $p=100,000$ parameters, the Jacobian would require $10^6 \times 10^5 \times 4$ bytes $\approx 400$ Gigabytes of memory. This is prohibitive.

**Conclusion and Path to True Scale**

The vectorized Jacobian method is a monumental improvement and is the correct approach for problems where the number of arms is in the hundreds or a few thousands, as in our textbook simulation.

For **millions of SKUs**, neither method is sufficient on its own. The K-Pass method is too slow, and the Jacobian method uses too much memory. Real-world large-scale bandit systems employ a hybrid, two-stage approach:

1.  **Candidate Generation (Retrieval):** A fast, approximate model (e.g., a two-tower model using embedding similarity, or even a simple business-rules engine) is used to select a small candidate set of a few hundred promising items from the millions available.
2.  **Re-Ranking:** The sophisticated `NeuralUCBAgent`, using the efficient vectorized `predict` method, is then applied *only to this small candidate set*.

This two-stage process combines the scalability of approximate retrieval with the principled exploration of a UCB algorithm. It is the dominant paradigm in industrial applications. For the remainder of our chapter, we will proceed with the superior vectorized implementation, keeping in mind that this candidate generation step is the necessary bridge to planet-scale systems.